In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize

__파일 경로__

In [ ]:
path='파일 위치'

__사용자 정의 stopwords__

In [ ]:
stopWords = set(stopwords.words('english'))
stopWords.update(['say','get','know','crosstalk','people','want','joe','would','think','go','much','that']) # for news

stopWords.update(['well','okay','yeah','right','thing','president','get','know','say']) # for debate

stopWords.update ["https","http","com","donald","trump","joe","biden","would", "election", "could", "also", "president","vote", "voters", "house", "run", "two", "rox", "reuters","way","go", "point", "take", "say"] # for twitter

__품사에 따른 표제어 추출 함수 재정의__

In [ ]:
WL = WordNetLemmatizer()

def proper_lemmatize(word,tag):
    
    if tag[0]=='V':
        ret= WL.lemmatize(word,'v')
    elif tag[0]=='J':
        ret= WL.lemmatize(word,'a')
    elif tag[0]=='NNP':
        ret= word
    else:
        ret= WL.lemmatize(word)
        
    return ret if ret not in stopWords else ' '

# 토론 데이터

데이터 로드

In [ ]:
first_debate = pd.read_csv(path+'us_election_2020_1st_presidential_debate.csv')
second_debate = pd.read_csv(path+'us_election_2020_2nd_presidential_debate.csv')
trump_town_hall = pd.read_csv(path+'us_election_2020_trump_town_hall.csv')
biden_town_hall = pd.read_csv(path+'us_election_2020_biden_town_hall.csv')

1 , 2차 대선 토론 및 연설 데이터 후보에 따라 통합

In [ ]:
first_Biden = first_debate[first_debate.speaker=='Vice President Joe Biden']
first_Trump = first_debate[first_debate.speaker=='President Donald J. Trump']

second_Biden = second_debate[second_debate.speaker=='Joe Biden']
second_Trump = second_debate[second_debate.speaker=='Donald Trump']

town_hall_Trump = trump_town_hall[trump_town_hall.speaker=='President Trump']
town_hall_Biden = biden_town_hall[biden_town_hall.speaker=='Joe Biden']

Trump_speaking = first_Trump.text.values.tolist()+second_Trump.text.values.tolist()+town_hall_Trump.text.values.tolist()
Biden_speaking = first_Biden.text.values.tolist()+second_Biden.text.values.tolist()+town_hall_Biden.text.values.tolist()

메인 코드

In [ ]:
Trump_speaking_token = []

for sentence in Trump_speaking:
    clean_words=[]
    sentence = re.sub('[^a-zA-Z-]+',' ',sentence)
    
    for word in word_tokenize(sentence):
        if len(word)>2 and word.lower() not in stopWords:
            if word not in ['Trump','Biden']:
                word = word.lower()
            clean_words.append(word)
    
    Trump_speaking_token.append(clean_words)
    
for i in range(len(Trump_speaking_token)):
    Trump_speaking_token[i] = [proper_lemmatize(word,tag) for (word,tag) in pos_tag(Trump_speaking_token[i])]
    
Trump_corpus = [*map(lambda x: ' '.join(x) , Trump_speaking_token)]

Biden_speaking_token = []

for sentence in Biden_speaking:
    clean_words=[]
    sentence = re.sub('[^a-zA-Z-]+',' ',sentence)
    
    for word in word_tokenize(sentence):
        if len(word)>2 and word.lower() not in stopWords:
            if word not in ['Trump','Biden']:
                word = word.lower()
            clean_words.append(word)
    
    Biden_speaking_token.append(clean_words)
    
for i in range(len(Biden_speaking_token)):
    Biden_speaking_token[i] = [proper_lemmatize(word,tag) for (word,tag) in pos_tag(Biden_speaking_token[i])]
    
Biden_corpus = [*map(lambda x: ' '.join(x) , Biden_speaking_token)]

Trump_corpus.to_csv('경로 포함 파일명')
Biden_corpus.to_csv('경로 포함 파일명')

# 뉴스 데이터

데이터 로드

In [ ]:
news = pd.read_csv(path+"news.csv")

기간 필터링 및 인덱스 초기화

In [ ]:
news = news.drop(news[(news["Date"] == "2020-11-01") | (news["Date"] == "2020-11-02")].index)
news = news.reset_index()
news = news.drop('index',axis=1)

데이터 프레임 상태로 전처리하기 위한 사용자 정의 함수

In [ ]:
def proper_tokenize(element):

    ret_list = []
    
    for sentence in element:
        clean_words=[]
        sentence = re.sub('[^a-zA-Z-]+',' ',sentence)

        for word in word_tokenize(sentence):
            if len(word)>2 and word.lower() not in stopWords:
                if word not in ['Trump','Biden']:
                    word = word.lower()
                clean_words.append(word)

        ret_list.append(clean_words)
    return ret_list

def proper_lemmatize(word,tag):
    
    if tag[0]=='V':
        ret= WL.lemmatize(word,'v')
    elif tag[0]=='J':
        ret= WL.lemmatize(word,'a')
    elif tag[0]=='NNP':
        ret= word
    else:
        ret= WL.lemmatize(word)
        
    return ret if ret not in stopWords else ''

def apply_lemmetize(element):
    
    for i in range(len(element)):
        element[i] = [proper_lemmatize(word,tag) for (word,tag) in pos_tag(element[i])]
        
    return element        

메인 코드

In [ ]:
news["Text"] = news["Text"].apply(lambda x: sent_tokenize(x))

news['Text'] = news['Text'].apply(proper_tokenize)
news['Text'] = news['Text'].apply(proper_tokenize)

news['Text'] = news['Text'].apply(apply_lemmetize)
news['Text'] = news['Text'].apply(apply_lemmetize)

news['Text'] = news['Text'].apply(lambda x : [*map(lambda y: ' '.join(y) , x)])
news['Text'] = news['Text'].apply(lambda x : [*map(lambda y: ' '.join(y) , x)])

new.to_csv('경로명 포함 파일명')

# 트위터

데이터 로드

In [ ]:
from_biden=pd.read_csv(path+"from_Biden_without_hashtag.csv")
to_biden=pd.read_csv(path+"to_Biden_without_hashtag.csv")
from_trump=pd.read_csv(path+"from_Trump_without_hashtag.csv")
to_trump=pd.read_csv(path+"to_Trump_without_hashtag.csv")

토큰화 , 역토큰화 진행 사용자 정의 함수

In [ ]:
def tokenizing(document):
    token = []
    for sentence in document:
        clean_words=[]
        sentence = re.sub('[^a-zA-Z-]+',' ',sentence)

        for word in word_tokenize(sentence):
            if len(word)>2 and word.lower() not in stopWords:
                if word not in ['Trump','Biden']:
                    word = word.lower()
                clean_words.append(word)
        token.append(clean_words)
    return token

def make_sent_corpus(lemmatized_token):
    sent_corpus = [*map(lambda x: ' '.join(x) , lemmatized_token)]
    return sent_corpus

메인 코드

In [ ]:
from_biden=from_biden.text.values.tolist()
to_biden=to_biden.text.values.tolist()
from_trump=from_trump.text.values.tolist()
to_trump=to_trump.text.values.tolist()

from_biden=tokenizing(from_biden)
to_biden=tokenizing(to_biden)
from_trump=tokenizing(from_trump)
to_trump=tokenizing(to_trump)

from_biden=lemmatizer(from_biden)
to_biden=lemmatizer(to_biden)
from_trump=lemmatizer(from_trump)
to_trump=lemmatizer(to_trump)

from_biden_corpus=make_sent_corpus(from_biden)
to_biden_corpus=make_sent_corpus(to_biden)
from_trump_corpus=make_sent_corpus(from_trump)
to_trump_corpus=make_sent_corpus(to_trump)

from_biden_corpus.to_csv('경로명 포함 파일명')
to_biden_corpus.to_csv('경로명 포함 파일명')
from_trump_corpus.to_csv('경로명 포함 파일명')
to_trump_corpus.to_csv('경로명 포함 파일명')